<a href="https://colab.research.google.com/github/FrozenPeak0701/MusicCopilotGPT/blob/main/suzukireproduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi -L
!lscpu |grep 'Model name'

GPU 0: Tesla T4 (UUID: GPU-c78fbd4c-82c3-4d48-bf5b-c2699d3184fb)
Model name:                      Intel(R) Xeon(R) CPU @ 2.20GHz


In [2]:
## Mount Google Drive Data (If using Google Colaboratory)
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
except:
    print("Mounting Failed.")

Mounted at /content/gdrive


In [3]:
!ls gdrive/MyDrive/Data/

ATEPP-1.1      detokenizer  requirements.txt	   tokenizer
convert.json   musicXml     sample.py		   tokenProduction.ipynb
CreateCsv.py   mxl	    test.ipynb
CSV	       __pycache__  tokenized_files
dataloader.py  remi	    tokenized_scores.json


In [4]:
data_dir = "gdrive/MyDrive/Data/"

In [5]:
!pip install -r "{data_dir}requirements.txt"
!pip install miditoolkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 51.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 7.3 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=e25cf2fecc44aff41a3953e8ac757ca28aecde8023f9922fc6986486dfc2419d
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi


In [6]:
import sys
sys.path.append(data_dir)

In [7]:
from dataloader import *
import pandas as pd
from remi import *
import miditoolkit
from tokenizer.score_to_tokens import *
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import json
import pickle
from torch.utils.data import Subset
from torch.optim import Adam
from torch.nn.utils.rnn import pad_sequence

In [8]:
torch.cuda.memory_allocated()

0

In [9]:
tokenized_file_dir = 'tokenized_files/'
dataset = CustomTokenDataset(f'{data_dir}CSV/data.csv')
tmp = dataset[0]

indices = list(range(len(dataset)))
td = [310, 308, 307, 306, 305, 304, 282, 280, 279, 278, 276, 264, 263, 256, 245, 242, 220, 213, 212, 211, 209, 207, 203, 196, 194, 193, 192, 191, 190, 186, 184, 182, 181 ,178, 177, 175, 174, 169, 66, 48, 33, 28, 27, 24]
for i in td:
  del indices[i]

new_dataset = Subset(dataset, indices)
len(new_dataset)

275

In [10]:
# tokenized_scores_list = ['sos', 'eos']
# max_midi_token_len = 0
# max_score_token_len = 0
# files = []
# for i,dat in enumerate(new_dataset):
#   print(f"{i + len(td)}|", end = '')
#   try:
#     midi_obj = miditoolkit.midi.parser.MidiFile(data_dir+dat[2])
#     if len(midi_obj.instruments) == 2:
#       if (midi_obj.instruments[0].name!="Piano") or ((midi_obj.instruments[1].name!="Piano")):
#         print(f"{i}th object's midi file isn't 2 pianos: {midi_obj.instruments[0].name}, {midi_obj.instruments[1].name}")
#       else:
#         print(f"{i}th object's midi file is 2 pianos")
#     elif len(midi_obj.instruments) == 1:
#       print(f"{i}th object's midi file is 1 piano: {midi_obj.instruments[0].name}")
#     else:
#       print(f"{i}th object's midi file is more than two piano: {[i.name for i in midi_obj.instruments]}")
#     max_midi_token_len = max(*[len(i.notes) for i in midi_obj.instruments], max_midi_token_len)
#     scoretokens = MusicXML_to_tokens(data_dir+dat[1])
#     for tokj in scoretokens:
#         if (tokj not in tokenized_scores_list):
#           tokenized_scores_list.append(tokj)
#     max_score_token_len = max(max_score_token_len, len(scoretokens))
#     files.append(dat[0])
#   except (TypeError, FileNotFoundError, AttributeError) as e:
#     print(f"{e}")
# with open(data_dir+tokenized_file_dir+'correspondance.json', 'w') as f:
#     json.dump(files, f)
# tokenized_scores_list, max_score_token_len, max_midi_token_len

In [11]:
# with open(data_dir+'tokenized_scores.json', 'w') as f:
#     json.dump(tokenized_scores_list, f)

with open(data_dir+'tokenized_scores.json') as f:
    tokenized_scores_list = json.load(f)

In [12]:
len(tokenized_scores_list)

422

In [13]:
def normalize(value, length):
    return value / length

def encode(note, song_length):
    start_token = int(normalize(note.start, song_length) * 100)
    end_token = int(normalize(note.end, song_length) * 100)
    pitch_token = note.pitch
    velocity_token = note.velocity
    return [start_token, end_token, pitch_token, velocity_token]

In [14]:
# for i, dat in enumerate(new_dataset):
#   song = miditoolkit.midi.parser.MidiFile(data_dir+dat[2])
#   song_length = song.max_tick
#   dnotes = sorted([j for i in song.instruments for j in i.notes], key=lambda note: note.start)
#   tokens = [encode(note, song_length) for note in dnotes ]
#   with open(data_dir + tokenized_file_dir+f'midi{i}.pkl', 'wb') as f:
#     pickle.dump(tokens, f)

In [15]:
# with open(data_dir + tokenized_file_dir+'midi1.pkl', 'rb') as f:
#     tokens = pickle.load(f)
# tokens

In [16]:
# Create a mapping from tokens to integers
token_to_int = {token: i for i, token in enumerate(tokenized_scores_list)}

# Create a mapping from integers to tokens
int_to_token = {i: token for token, i in token_to_int.items()}

def token2intlist(tokens):
  return [token_to_int[i] for i in tokens]
def int2tokenlist(ints):
  return [int_to_token[i] for i in ints]

In [17]:
# for i, dat in enumerate(new_dataset):
#   strlist = MusicXML_to_tokens(data_dir+dat[1])
#   strlist = ['sos',] + strlist + ['eos',]
#   tokens = token2intlist(strlist)
#   with open(data_dir + tokenized_file_dir+f'score{i}.pkl', 'wb') as f:
#     pickle.dump(tokens, f)

In [18]:
# with open(data_dir + tokenized_file_dir+f'score0.pkl', 'rb') as f:
#     tokens = pickle.load(f)
# tokens

In [19]:
srcr = []
tgtr = []
max_src_len = 0
max_tgt_len = 0
for i in range(275):
  with open(data_dir + tokenized_file_dir+f'score{i}.pkl', 'rb') as f:
      tok = pickle.load(f)
      max_tgt_len = max(max_tgt_len, len(tok))
      tgtr.append(tok)

  with open(data_dir + tokenized_file_dir+f'midi{i}.pkl', 'rb') as f:
      tok = pickle.load(f)
      max_src_len = max(max_src_len, len(tok))
      srcr.append(tok)
len(srcr), len(tgtr), max_src_len, max_tgt_len

(275, 275, 10188, 35382)

In [20]:
len(tgtr[0]), len(srcr[0])

(4976, 1288)

In [21]:
class MusicDataset(Dataset):
    def __init__(self, src_data, tgt_data):
        self.src_data = src_data
        self.tgt_data = tgt_data

    def __len__(self):
        return len(self.src_data)

    def __getitem__(self, idx):
        return self.src_data[idx], self.tgt_data[idx]

def pad_sequences(batch, padding_value=-1, feature_size = None):
    # Get sequence lengths
    lengths = [len(seq) for seq in batch]

    # Create an empty tensor with size of the longest sequence
    max_length = max(lengths)
    if feature_size is None:
      padded_batch = torch.full((len(batch), max_length), padding_value)
    else:
      padded_batch = torch.full((len(batch), max_length, feature_size), padding_value)

    # Pad the sequences
    for i, seq in enumerate(batch):
        padded_batch[i, :lengths[i]] = seq

    return padded_batch

def collate_fn(batch):
    src_batch, tgt_batch = zip(*batch)

    src_batch = pad_sequences([torch.tensor(data) for data in src_batch],feature_size = 4)
    tgt_batch = pad_sequences([torch.tensor(data) for data in tgt_batch])

    return src_batch, tgt_batch

# Assuming your data is stored in src_data and tgt_data
dataset = MusicDataset(srcr, tgtr)

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=2, collate_fn=collate_fn)  #, shuffle = True

In [22]:
class Transformer(nn.Module):
    def __init__(self, embedding_size, num_heads, hidden_dim, num_layers, dropout, device, max_src_length=11000, max_tgt_length = 36000):
        super().__init__()

        self.device = device

        self.src_linear = nn.Linear(4, embedding_size)
        self.tgt_linear = nn.Linear(1, embedding_size)
        self.src_pos_embedding = nn.Embedding(max_src_length, embedding_size)
        self.tgt_pos_embedding = nn.Embedding(max_tgt_length, embedding_size)

        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(embedding_size, num_heads, hidden_dim, dropout),
            num_layers
        )

        self.decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(embedding_size, num_heads, hidden_dim, dropout),
            num_layers
        )

        self.fc = nn.Linear(embedding_size, 1)
        self.dropout = nn.Dropout(dropout)

    def create_mask(self, src, tgt):
      src_mask = (src.sum(dim=-1) == -4)
      tgt_mask = (tgt == -1)
      print(src_mask,tgt_mask, sep = '\n')
      # # This line ensures that the `inf` values do not result in NaN values after softmax
      # src_mask = src_mask.masked_fill(src_mask, sys.float_info.min)
      # tgt_mask = tgt_mask.masked_fill(tgt_mask, sys.float_info.min)
      # print(src_mask,tgt_mask, sep ='\n')
      return src_mask, tgt_mask


    def forward(self, src, tgt):
      print(1,torch.cuda.memory_allocated()*1e-9)
      src = src.float()
      tgt = tgt.float()

      batch_size, src_seq_len, _ = src.size()
      _, tgt_seq_len = tgt.size()  # get sequence length of tgt
      print(2,torch.cuda.memory_allocated()*1e-9)
      src_mask, tgt_mask = self.create_mask(src, tgt)

      print(src_mask.shape, tgt_mask.shape,torch.cuda.memory_allocated()*1e-9)

      pos = (torch.arange(0, src_seq_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)).long()
      tgt_pos = (torch.arange(0, tgt_seq_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)).long()  # pos for tgt
      print(pos.shape, tgt_pos.shape)
      print(3,torch.cuda.memory_allocated()*1e-9)
      # print((self.src_linear(src) + self.src_pos_embedding(pos)).shape, (self.tgt_linear(tgt.unsqueeze(-1)) + self.tgt_pos_embedding(tgt_pos)).shape)
      src_embedded = self.dropout((self.src_linear(src) + self.src_pos_embedding(pos)).permute(1,0,2))
      tgt_embedded = self.dropout((self.tgt_linear(tgt.unsqueeze(-1)) + self.tgt_pos_embedding(tgt_pos)).permute(1,0,2))

      print(4,torch.cuda.memory_allocated()*1e-9)
      print(src_mask.shape, tgt_mask.shape, src_embedded.shape, tgt_embedded.shape)
      enc_src = self.encoder(src_embedded, src_key_padding_mask=src_mask)
      print(5,torch.cuda.memory_allocated()*1e-9)
      # print(src_mask, tgt_mask, src_mask.shape, tgt_mask.shape)
      print(enc_src.shape)
      output = self.decoder(tgt_embedded, enc_src, tgt_key_padding_mask=tgt_mask, memory_key_padding_mask=src_mask)
      print(6,torch.cuda.memory_allocated()*1e-9)
      return self.fc(output.permute(1, 0, 2))

In [23]:
# Initialize model
embedding_size = 64
num_heads = 4
hidden_dim = 64
num_layers = 2
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Transformer(embedding_size, num_heads, hidden_dim, num_layers, dropout, device).to(device)

# Initialize optimizer and loss function
learning_rate = 0.0001
optimizer = Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

In [24]:
model

Transformer(
  (src_linear): Linear(in_features=4, out_features=64, bias=True)
  (tgt_linear): Linear(in_features=1, out_features=64, bias=True)
  (src_pos_embedding): Embedding(11000, 64)
  (tgt_pos_embedding): Embedding(36000, 64)
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=64, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=64, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
    )
  )
  (decoder): TransformerDecoder(
    (layers): ModuleList(
      (0-1)

In [25]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,142,913 trainable parameters


In [26]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for i, (src, tgt) in enumerate(dataloader):
      # if i==0:
        # Send your batch to the correct device
        # torch.cuda.empty_cache()

        optimizer.zero_grad()
        src = src.to(device).float()
        tgt = tgt.to(device).float()

        print('src shape and tgt shape:',src.shape,tgt.shape)
        # Pass data through model
        # The input to the model is the source sequences only
        output = model(src, tgt)

        # print(output, output.shape, tgt.shape)
        # Compute loss
        loss = criterion(output.squeeze(-1), tgt)
        print(7, torch.cuda.memory_allocated()*1e-9)
        # Backward pass and optimization

        loss.backward()
        optimizer.step()

        if i % 1 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.detach().item()}')

# Save model
# torch.save(model.state_dict(), "transformer_model.pt")

src shape and tgt shape: torch.Size([2, 2930, 4]) torch.Size([2, 7942])
1 0.012741632000000001
2 0.012741632000000001
tensor([[False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ..., False, False, False]], device='cuda:0')
tensor([[False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ..., False, False, False]], device='cuda:0')
torch.Size([2, 2930]) torch.Size([2, 7942]) 0.01276416
torch.Size([2, 2930]) torch.Size([2, 7942])
3 0.012938752000000001
4 0.028417024000000003
torch.Size([2, 2930]) torch.Size([2, 7942]) torch.Size([2930, 2, 64]) torch.Size([7942, 2, 64])
5 1.3055129600000002
torch.Size([2930, 2, 64])
6 13.891348480000001
7 13.885887488000002


OutOfMemoryError: ignored